In [1]:
import pandas as pd
import json


import transformers
from alphabet_detector import AlphabetDetector


In [2]:
import torch

In [5]:
torch.device('cuda')

device(type='cuda')

In [2]:
ad = AlphabetDetector()

data_path = 'eng_rus_data(2).csv'
model_checkpoint = "experiment_2/checkpoint-14000"
batch_size = 8

In [3]:
df = pd.read_csv(data_path, sep='\t')

In [4]:
df = df[['tags', 'tokens']]

In [5]:
df["tags"] = df["tags"].apply(eval)
df["tokens"] = df["tokens"].apply(eval)

In [6]:
# df = pd.DataFrame([json.loads(l) for l in open(data_path)])
df = df[df.tags.apply(len) != 0]
df['tags'] = df.tags.apply(lambda x: None if isinstance(x[0], list) else x)
df = df.dropna()

In [7]:
df.tags

0        [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
1                 [O, O, O, O, O, O, O, O, O, O, B-SYM, O]
2                                                [O, O, O]
3                                       [O, O, O, O, O, O]
4                                 [O, O, O, O, O, O, O, O]
                               ...                        
53457    [O, O, O, O, O, B-SYM, O, B-SYM, I-SYM, O, O, ...
53458    [O, O, O, O, B-ILL, I-ILL, O, O, O, O, O, O, O...
53459    [O, O, O, O, O, O, B-SYM, I-SYM, O, O, O, O, O...
53460    [O, O, O, B-ILL, I-ILL, O, O, O, O, O, O, O, O...
53461                          [O, O, O, O, O, O, O, O, O]
Name: tags, Length: 53438, dtype: object

In [ ]:
df['tags'] = df.tags.apply(lambda x: None if isinstance(x[0], list) else x)

In [ ]:
df.tags

In [8]:
df = df.to_dict(orient='index')

In [11]:
data = []
for v in df.values():
    add = True
    for t in v['tags']:
        if t not in uniq:
            add = False
    if add:
        data.append(v)
    

In [12]:
df = pd.DataFrame(data)

In [10]:
uniq = {'O',
        'B-ILL',
        'B-SYM',
        'I-ILL',
        'I-SYM'}

# df[df.tags.apply(lambda x: [t for t in x if type(t) i])]

In [13]:
# df = df[df.tokens.apply(lambda x: ad.is_cyrillic(' '.join(x)))]

df['len_c'] = df['tokens'].apply(lambda x: len(' '.join(x)))
df['len_t'] = df['tokens'].apply(lambda x: len(x))

df = df[df.len_c < 400]
df = df[df.len_t < 50]
df = pd.concat([df[df.tags.apply(set) != {'O'}], df[df.tags.apply(set) == {'O'}].sample(4000)])

In [14]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.15, random_state=42)



In [15]:
len(df_train), len(df_test)

(21967, 3877)

In [16]:
label_list = sorted(set([item for sublist in df["tags"].values for item in sublist]))
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O', 'B-ILL', 'B-SYM', 'I-ILL', 'I-SYM']

In [17]:
from datasets import Dataset, DatasetDict

ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(df_train)),
    'test': Dataset.from_pandas(pd.DataFrame(df_test))
})
ner_data



DatasetDict({
    train: Dataset({
        features: ['tags', 'tokens', 'len_c', 'len_t', '__index_level_0__'],
        num_rows: 21967
    })
    test: Dataset({
        features: ['tags', 'tokens', 'len_c', 'len_t', '__index_level_0__'],
        num_rows: 3877
    })
})

In [18]:
from datasets import load_dataset, load_metric

In [19]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
example = df_train.iloc[7]
print(example["tokens"])
print(example["tags"])

In [ ]:
tokenized_input = tokenizer([t.lower() for t in example["tokens"]], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print(tokenized_input.word_ids())

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

In [20]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs



In [ ]:
tokenize_and_align_labels(ner_data['train'][22:23])

In [21]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/4 [00:00<?, ?ba/s]

In [22]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [23]:
args = TrainingArguments(
    output_dir='experiment_2/',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy='epoch',
    report_to='none',
)

In [24]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [25]:
metric = load_metric("seqeval")

In [26]:
example = df_train.iloc[10]
labels = example['tags']
print(example['tokens'])
print(example['tags'])
print()
metric.compute(predictions=[labels], references=[labels])



['Для', 'профилактики', 'добавляю', 'в', 'аромалампу', 'смесь', 'натуральных', 'эфирных', 'масел', ',', 'при', 'первых', 'признаках', 'простуды', 'добавляю', 'эту', 'смесь', 'в', 'тепловлажный', 'ингалятор', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ILL', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



{'ILL': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [27]:


import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }



In [28]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [29]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [ ]:
trainer.save?

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.057000,0.364178,0.890837,0.887611,0.889221,0.929216
2,0.027600,0.412191,0.888380,0.900724,0.894509,0.931686


Configuration saved in experiment_2/checkpoint-2746/config.json
Model weights saved in experiment_2/checkpoint-2746/pytorch_model.bin
tokenizer config file saved in experiment_2/checkpoint-2746/tokenizer_config.json
Special tokens file saved in experiment_2/checkpoint-2746/special_tokens_map.json
Configuration saved in experiment_2/checkpoint-5492/config.json
Model weights saved in experiment_2/checkpoint-5492/pytorch_model.bin
tokenizer config file saved in experiment_2/checkpoint-5492/tokenizer_config.json
Special tokens file saved in experiment_2/checkpoint-5492/special_tokens_map.json


TrainOutput(global_step=5492, training_loss=0.04387896493326897, metrics={'train_runtime': 459.0522, 'train_samples_per_second': 95.706, 'train_steps_per_second': 11.964, 'total_flos': 1091238535574460.0, 'train_loss': 0.04387896493326897, 'epoch': 2.0})

In [31]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ILL': {'precision': 0.8401264933239635,
  'recall': 0.8694545454545455,
  'f1': 0.8545389563974268,
  'number': 2750},
 'SYM': {'precision': 0.8960617587827254,
  'recall': 0.9055857078245138,
  'f1': 0.9007985603419189,
  'number': 17688},
 'overall_precision': 0.8883795000482579,
 'overall_recall': 0.9007241413054115,
 'overall_f1': 0.8945092322643343,
 'overall_accuracy': 0.9316863587540279}

In [32]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [33]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ILL,B-SYM,I-ILL,I-SYM
O,44923,146,871,55,1033
B-ILL,134,2420,123,39,31
B-SYM,761,146,16294,0,474
I-ILL,166,52,9,711,147
I-SYM,914,68,463,92,13718


In [34]:
from transformers import pipeline

In [35]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [36]:
pipe('Заложенность носа сохраняется в любое время года, но усиливается во время сна. около полугода назад появился ')

[{'entity_group': 'SYM',
  'score': 0.9996667,
  'word': 'заложенность носа',
  'start': 0,
  'end': 17}]

In [37]:
import razdel
from spacy import displacy
from IPython.core.display import display, HTML

In [38]:
def display_predictions(text):
    sents = razdel.sentenize(text)

    for sent in sents:
        
        ents = [{'start': ent['start'],
                'end': ent['end'],
                'label': ent['entity_group']} for ent in pipe(sent.text.lower())]
        ex = {"text": sent.text,
            "ents": ents}

        colors = {"SYM": "linear-gradient(90deg, #aa9cfc, #fc9ce7)",
                  "ILL": "linear-gradient(90deg, rgba(122,153,180,1) 0%, rgba(5,248,141,1) 53%, rgba(0,212,255,1) 100%)"}
        html = displacy.render(ex, style="ent", manual=True, options={'colors':colors});
        display(HTML(html));

In [39]:
text = """"Здравствуйте Евгений Альбертович! нам очень рекомендовали. Вы наша последняя надежда! моему сыну 3 года, из них дышит без капель года полтора. Заложенность носа сохраняется в любое время года, но усиливается во время сна. около полугода назад появился неприятный запах из носоглотки. Во время простуды отделяемое из носа очень густое и плохо отходит. Находится слизь глубоко, ближе к глотке. были у лора, ни полипов, ни аденоид нет. Отправили нас к аллергологу, там нам назначили "сингуляр" и "авамис". Ни чего из этого нам не помогло. делали бак. посев из зева и носа. В зеве высеяли st.aureus умеренный рост и acinotobakter. Лечились комплексным бактериофагом, помогло примерно на неделю. на праздниках мы заболели и как осложнение отит. Лечились зинатом. На фоне этого лечения практически прошла заложенность носа. Но после отмены все началось снова. Очень хочется попасть к вам на прием."""
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

...

/home/merionum/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:906: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [40]:
text = 'Вечно не хватает сна. Часто не могу уснуть, но даже если сплю больше 8 часов, всеравно просыпаюсь уставшая. Не хочу вставать, будто нет сил что то делать. Постоянно наедаюсь'
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [41]:
text = "Сегодня мне нравится произносить слова и не гундосить. Совсем недавно я избегала с кем-либо лишний раз разговаривать, потому что звучало некрасиво говорить в нос. Обоняние пропало два года назад. Дышать получалось только ртом, потому что нос был забит и как обнаружилось в носу выросли полипы, которые распространялись в преддверие носа и в носоглотку. Не соглашалась с мыслью об операции пока не увидела в зеркале уже видимый из носа полип. Сейчас я хожу с закрытым ртом, дышу носом и полной грудью, и не могу нарадоваться этому своему иному качественному состоянию. Вернулось обоняние. Подарил мне это чудо свободного дыхания очень профессиональный доктор, ювелирных дел мастер, врач с большой буквы Гилифанов Евгений Альбертович. Операция по удалению полипов, остеомы решетчатого лабиринта длилась около пяти часов в Клинической больнице № 1. Спасибо вам, Евгений Альбертович, за ваши золотые руки, за высокий профессионализм и за ваше чуткое отношение. Крепкого вам здоровья и успехов в вашем нелегком труде! Я очень вам благодарна"
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

...

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [42]:
text = "при осмотре грудня клетка астенической формы, симметрична,при пальпации эластична, голосовое дрожание усилено слева в верхних отделах,при перкуссии там же звук с коробочным оттенком,при аускультации дыхание амфорическое"
display_predictions(text)

<IPython.core.display.HTML object>

In [43]:
text = """Добрый день, подскажите признаки энтеробиоза кишечника? какие анализы сдаются для подтверждения диагноза?Могут быть гранулемы терминального отдела в тонком кишечнике - причиной энтеробиоза? Болезнь крона не подтвердили по гистологии, сахоромицеты-ОТРИЦАТЕЛЬНО,кальпротектин-ОТРИЦАТЕЛЬНО. В кале постоянно присутствует желтая слизь и беспокоят растройства кишечника(болей нет)."""
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [44]:
text = """Здравствуйте! Поражение обоняния при коронавирусной инфекции хорошо известный факт. Отсутствие или частичное восстановление обоняния восстанавливается медленно, растягивается до 6 месяцев. Занимаются этим ЛОР-врачи. Во Владивостоке эту проблему решает Гилифанов Евгений Альбертович, заведующий кафедрой ЛОР-болезней медицинского университета. Обратитесь к нему. Он отвечает на сайте портала. С уважением, проф. Попов А.Ф."""
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [45]:
text = """Здравствуйте! Александр Фёдорович , у моего мужа сладж желчного пузыря , дискенизия и полипы под подозрением . Практически постоянно тянущая ,ноющая боль в районе желчного. Проходит лечение ( урсофальк , одестон ). Улучшений нет . Диспепсия ,иногда изжога,похудел, вес не набирается , на языка желтый налёт , быстро утомляется . Эозинофилы : 4,2 % . До этого были 3% . Лимфоциты : 53,6% . Моноциты : 8,9%. Подскажите могут ли такие симптомы давать гельминты , вирусы?  Какие анализы посоветуете сдать ? Благодарю за ответ !"""
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

...

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [46]:
text = "Боль в пояснице, между лопаток, утомляемость, боль в паху справа"
display_predictions(text)

<IPython.core.display.HTML object>

In [47]:
text = "температура порядка 38, сбивается нурофеном, потом снова растет; голова \"тяжелая\", знобит"
display_predictions(text)

<IPython.core.display.HTML object>

In [48]:
text = "Давление в грудной клетке, головная боль, боль в груди при вдохе, постоянная температура 37,3, температура держится 3 месяца"
display_predictions(text)

<IPython.core.display.HTML object>

In [49]:
text = "Болит челюсти справа снизу и сверху, отдает головной болью в висок, глазницу и ухо, боль очень острая и сильная"
display_predictions(text)

<IPython.core.display.HTML object>

In [50]:
text = "Дорсопатия. Дегенеративный спондилолистез L4-L5. Вторичный стеноз позвоночного канала на уровне L4-L5.  Диабетическая полинейропатия.  Болезнь Рота.  Остеопороз тел позвоенков поясничного отдела позвоночника."
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [51]:
text = "Миома  матки,  смешанная  форма.  Нарушение  менструального  цикла,  метроррагия.  17.06.19 Диагностическая  лапароскопия. Лапаротомия, надвлагалищная ампутация  матки с  левой маточной  трубой."
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [52]:
text = "Застарелое повреждение внутренних менисков обоих коленных суставов. Двусторонний гонартроз 1 ст."
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [53]:
text = "Нестенозирующий атеросклероз БЦА. Функциональное расстройство нервной системы.  астено-невротическое состояние,диссомния, стабилизация на терапии   Односторонний пульсирующий тиннитус , регресс"
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [54]:
text = "функциональная диспесия. синдром абдоминальной боли.  Хронический гастрит, обострение?  Дискинезия желчевыводящих путей, дисфункция желчного пузыря по гипокинетическому типу.  Дисбиоз кишечника. Флатуленция"
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [55]:
text = "Дорсопатия. Начальные проявления ДДЗП. Грыжи дисков, Люмбалгия, хроническое течение, обострение. Мышечно-тонический синдром в грудном и поясничном отделах позвоночника"
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [56]:
text = "ОРВИ. Острый ринофарингит. ИМВП"
display_predictions(text)

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

In [89]:
text = """"""
display_predictions(text)

<IPython.core.display.HTML object>